In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Loading data
data=pd.read_csv("uomds20/train.csv")

In [3]:
# Below print statements used to identify missing values
print(data.isnull().sum())

tripid                         0
additional_fare              202
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64


In [4]:
# Drop the rows which are included null values
data = data.dropna()

In [5]:
# Lable value map to 0 and 1
data['label'] = data['label'].map({"correct": 1, "incorrect": 0})

In [6]:
# Devide dataset to feature and label set
X,y =data.iloc[:,~data.columns.isin(['tripid','pickup_time', 'drop_time','label','pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'meter_waiting_till_pickup', 'additional_fare'])],data['label']

In [7]:
#split dataset to train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=562)

In [8]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [9]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.81%


In [10]:
testData = pd.read_csv("uomds20/test.csv")

In [12]:
finalDf = testData[['tripid']]
test_x =testData.iloc[:,~testData.columns.isin(['tripid','pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'meter_waiting_till_pickup', 'additional_fare'])]
pred = model.predict(test_x)
finalDf["prediction"] = pred

/home/dinesh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
finalDf.head(10)

,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1
5,213299545,0
6,213302332,1
7,213302671,1
8,213305594,1
9,213305134,1


In [14]:
finalDf.to_csv("submission.csv", index= False)